# GTSF phase I: biomass prediction

In this notebook, we are forecasting the weights by finding the closest blender model

### Look at the volumes created with blender

Load blender data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

import glob
import os
import boto3
import tempfile
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.stats import norm
import tqdm
import pickle
from itertools import combinations

from PIL import Image, ImageDraw
from multiprocessing import Pool, Manager
import copy
import uuid
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', 500)





<h1> Get world keypoint coordinates from GTSF data </h1>

In [ ]:
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")


sql_credentials = json.load(open(os.environ["SQL_CREDENTIALS"]))
sql_engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                           sql_credentials["host"], sql_credentials["port"],
                           sql_credentials["database"]))

Session = sessionmaker(bind=sql_engine)
session = Session()

Base = automap_base()
Base.prepare(sql_engine, reflect=True)
Enclosure = Base.classes.enclosures
Calibration = Base.classes.calibrations
GtsfDataCollection = Base.classes.gtsf_data_collections
StereoFramePair = Base.classes.stereo_frame_pairs


<h1> Utility functions for world keypoint normalization </h1>

In [ ]:
def generate_rotation_matrix(u_base, v):
    u = v / np.linalg.norm(v)
    n = np.cross(u_base, u)
    n = n / np.linalg.norm(n)
    theta = -np.arccos(np.dot(u, u_base))

    R = np.array([[
        np.cos(theta) + n[0]**2*(1-np.cos(theta)), 
        n[0]*n[1]*(1-np.cos(theta)) - n[2]*np.sin(theta),
        n[0]*n[2]*(1-np.cos(theta)) + n[1]*np.sin(theta)
    ], [
        n[1]*n[0]*(1-np.cos(theta)) + n[2]*np.sin(theta),
        np.cos(theta) + n[1]**2*(1-np.cos(theta)),
        n[1]*n[2]*(1-np.cos(theta)) - n[0]*np.sin(theta),
    ], [
        n[2]*n[0]*(1-np.cos(theta)) - n[1]*np.sin(theta),
        n[2]*n[1]*(1-np.cos(theta)) + n[0]*np.sin(theta),
        np.cos(theta) + n[2]**2*(1-np.cos(theta))
    ]])
    
    return R

In [ ]:
def euclidean_distance(p1, p2):
    if type(p1) == list:
        p1 = np.array(p1)
    if type(p2) == list:
        p2 = np.array(p2)
    return np.linalg.norm(p1 - p2)


def normalize_world_keypoints(wkps):
    body_parts = wkps.keys()
    wkps = {bp: np.array(wkps[bp]) for bp in body_parts}
    
    # translate keypoints such that tail notch is at origin
    translated_wkps = {bp: wkps[bp] - wkps['TAIL_NOTCH'] for bp in body_parts}
    
    # perform first rotation
    u_base=np.array([1, 0, 0])
    v = translated_wkps['UPPER_LIP']
    R = generate_rotation_matrix(u_base, v)
    norm_wkps_intermediate = {bp: np.dot(R, translated_wkps[bp]) for bp in body_parts}
    
    # perform second rotation
    u_base = np.array([0, 0, 1])
    v = norm_wkps_intermediate['DORSAL_FIN'] - np.array([norm_wkps_intermediate['DORSAL_FIN'][0], 0, 0])
    R = generate_rotation_matrix(u_base, v)
    norm_wkps = {bp: np.dot(R, norm_wkps_intermediate[bp]) for bp in body_parts}
    
    # perform reflecton if necessary
    if norm_wkps['PECTORAL_FIN'][1] > 0:
        norm_wkps = {bp: np.array([
            norm_wkps[bp][0],
            -norm_wkps[bp][1],
            norm_wkps[bp][2]
        ]) for bp in body_parts}
    
    return norm_wkps
    



<h1> Utility Method: World Keypoint Calculation </h1>

In [ ]:
# DEFINE OPTICAL PROPERTIES

# all distance are in meters
FOCAL_LENGTH = 0.00843663
BASELINE = 0.128096
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01035
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000

def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = PIXEL_COUNT_WIDTH / 2.0  
    image_center_y = PIXEL_COUNT_HEIGHT / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / PIXEL_COUNT_WIDTH)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / PIXEL_COUNT_HEIGHT)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])

def depth_from_disp(disp):
    """ calculate the depth of the point based on the disparity value """
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth

def disp_from_depth(depth):
    disp = FOCAL_LENGTH_PIXEL * BASELINE / depth
    return disp


<h1> Generate accuracy metrics on GTSF data </h1>

In [ ]:
session.rollback()

<h1> Train linear model with PCA + interaction features </h1>

In [ ]:
sfps_all = session.query(StereoFramePair).all()
df = pd.DataFrame()

body_parts = sorted([
    'TAIL_NOTCH',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'DORSAL_FIN',
    'UPPER_LIP',
    'EYE'
])

session.rollback()
for idx, row in enumerate(sfps_all):
    if idx % 10 == 0:
        print(idx)
        
    # get fish_id and ground truth metadata
    if row.gtsf_fish_identifier == '190321010002':
        continue
    ground_truth_metadata = json.loads(row.ground_truth_metadata)
    if ground_truth_metadata['data'].get('species') != 'salmon':
        continue
    
    left_keypoints = json.loads(row.left_image_keypoint_coordinates)
    right_keypoints = json.loads(row.right_image_keypoint_coordinates)
    wkps = json.loads(row.world_keypoint_coordinates)

    df_row = {'0': idx}
    for i in range(len(body_parts)-1):
        for j in range(i+1, len(body_parts)):
            d = euclidean_distance(wkps[body_parts[i]], wkps[body_parts[j]])
            df_row['{0}-{1}'.format(i, j)] = d
    
    if 'weight' not in ground_truth_metadata['data'].keys():
        continue
    weight = ground_truth_metadata['data']['weight']
    length = ground_truth_metadata['data']['length']
    width = ground_truth_metadata['data']['width']
    breadth = ground_truth_metadata['data']['breath']
    df_row['weight'] = weight
    df_row['length'] = length
    df_row['width'] = width
    df_row['breadth'] = breadth
    df_row['kfactor'] = 1e5 * weight / length**3
    df_row['date'] = row.date
    df_row['project_name'] = row.annotations_project_name
    df_row['left_keypoints'] = json.loads(row.left_image_keypoint_coordinates)
    df_row['right_keypoints'] = json.loads(row.right_image_keypoint_coordinates)
    df_row['world_keypoints'] = wkps
    df_row['gtsf_fish_identifier'] = row.gtsf_fish_identifier
    df_row['epoch'] = row.epoch
        
    
    
    df = df.append(df_row, ignore_index=True)
            
    



<h1> Apply filters </h1>

In [ ]:
df_cache = df.copy()
df = df.dropna()

In [ ]:
df = df_cache.copy()

In [ ]:
def coord2biomass_linear(world_keypoints, model):
    """from coordinates to biomass"""

    mean = model['mean']
    std= model['std']
    PCA_components = model['PCA_components']
    reg_coef = model['reg_coef']
    reg_intercept = model['reg_intercept']
    body_parts = model['body_parts']
    print(body_parts)
    # calculate pairwise distances for production coord
    # based on the exact ordering reflected in the body_parts
    # variable above

    pairwise_distances = []
    for i in range(len(body_parts)-1):
        for j in range(i+1, len(body_parts)):
            dist = euclidean_distance(world_keypoints[body_parts[i]], world_keypoints[body_parts[j]])
            pairwise_distances.append(dist)

    interaction_values_quadratic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            dist1 = pairwise_distances[i]
            dist2 = pairwise_distances[j]
            interaction_values_quadratic.append(dist1 * dist2)

    interaction_values_cubic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            for k in range(j, len(pairwise_distances)):
                dist1 = pairwise_distances[i]
                dist2 = pairwise_distances[j]
                dist3 = pairwise_distances[k]
                interaction_values_cubic.append(dist1 * dist2 * dist3)


    X = np.array(pairwise_distances + interaction_values_quadratic + interaction_values_cubic)

    X_normalized = (X - model['mean']) / model['std']
    X_transformed = np.dot(X_normalized, model['PCA_components'].T)
    prediction = np.dot(X_transformed, reg_coef) + reg_intercept
    return prediction

In [ ]:
def apply_filters(left_keypoints, right_keypoints, world_keypoints, baseline_biomass_model):
    filter_out, reason = False, None
    
    # apply y-coordinate deviation filter
    body_parts = sorted(list(left_keypoints.keys()))
    max_y_coordinate_deviation = max([abs(left_keypoints[bp][1] - right_keypoints[bp][1]) for bp in body_parts])
    max_x_coordinate_deviation = max([abs(left_keypoints[bp][0] - right_keypoints[bp][0]) for bp in body_parts])
    print(max_y_coordinate_deviation, max_x_coordinate_deviation)
    if (max_y_coordinate_deviation > 25):
        filter_out = True
        reason = 'Y-coordinate deviation too high'
        
    # apply world y-coordinate deviation filter
    norm_wkps = normalize_world_keypoints(world_keypoints)
    y_world_coordinates = [norm_wkps[bp][1] for bp in body_parts]
    max_y_world_coordinate_deviation = max(y_world_coordinates) - min(y_world_coordinates)
    if max_y_world_coordinate_deviation > 0.25:
        filter_out = True
        reason = 'World y-coordinate deviation too high'
        
    # apply baseline biomass model
    baseline_weight_prediction = coord2biomass_linear(world_keypoints, baseline_biomass_model)
    if (baseline_weight_prediction < 0) or (baseline_weight_prediction > 15000):
        filter_out = True
        reason = 'Baseline prediction way too off'
        
    
    return filter_out, reason


In [ ]:
baseline_biomass_model = pickle.load(open('/root/data/alok/biomass_estimation/models/model_v2.pkl', 'rb'))
df['filter_out'] = False
df['reason'] = None
for idx, row in df.iterrows():
    filter_out, reason = \
        apply_filters(row.left_keypoints, row.right_keypoints, row.world_keypoints, baseline_biomass_model)
    if filter_out:
        df.at[idx, 'filter_out'] = True
        df.at[idx, 'reason'] = reason

    

In [ ]:
df = df[(df.project_name != 'Automated keypoints detection') & (df.weight != 5057.0)]

In [ ]:
def generate_train_mask(df, train_frac, randomize=True):
    x = np.zeros((df.shape[0]), dtype=bool)
    x[:int(train_frac * df.shape[0])] = True
    np.random.shuffle(x)
    mask = pd.Series(x)
    return x
    


In [ ]:
# define all features

body_parts = sorted([
    'TAIL_NOTCH',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'DORSAL_FIN',
    'UPPER_LIP',
    'EYE'
])

pairwise_distance_columns = ['{0}-{1}'.format(x, y) for x, y in list(combinations(list(range(len(body_parts))), 2))]
interaction_columns_quadratic = []
interaction_columns_cubic = []
for i in range(len(pairwise_distance_columns)):
    for j in range(i, len(pairwise_distance_columns)):
        col1 = pairwise_distance_columns[i]
        col2 = pairwise_distance_columns[j]
        interaction_column = '{},{}'.format(col1, col2)
        df[interaction_column] = df[col1] * df[col2]
        interaction_columns_quadratic.append(interaction_column)
        
for i in range(len(pairwise_distance_columns)):
    for j in range(i, len(pairwise_distance_columns)):
        for k in range(j, len(pairwise_distance_columns)):
            col1 = pairwise_distance_columns[i]
            col2 = pairwise_distance_columns[j]
            col3 = pairwise_distance_columns[k]
            interaction_column = '{},{},{}'.format(col1, col2, col3)
            df[interaction_column] = df[col1] * df[col2] * df[col3]
            interaction_columns_cubic.append(interaction_column)

In [ ]:
np.random.seed(0)

mask = generate_train_mask(df, train_frac=0.8)
mask = mask & (df.index != 830) #& (~df.gtsf_fish_identifier.str.contains('viking')) 
columns = pairwise_distance_columns + interaction_columns_quadratic + interaction_columns_cubic

X_train = df.loc[mask, columns].values
y_train = df.loc[mask, 'weight'].values
X_test = df.loc[~mask, columns].values
y_test = df.loc[~mask, 'weight'].values

scaler = StandardScaler()
scaler.fit(X_train)
X_train_normalized = scaler.transform(X_train)

pca = PCA(n_components=min(X_train_normalized.shape[0], X_train_normalized.shape[1]))
pca.fit(X_train_normalized)
explained_variance_ratio = pca.explained_variance_ratio_.cumsum()
# idx = np.where(explained_variance_ratio > 0.999)[0][0]
idx = 4
print(idx)

pca = PCA(n_components=idx+1)
pca.fit(X_train_normalized)
X_train_transformed = pca.transform(X_train_normalized)
X_test_normalized = scaler.transform(X_test)
X_test_transformed = pca.transform(X_test_normalized)

reg = LinearRegression().fit(X_train_transformed, y_train)
print(reg.score(X_test_transformed, y_test))

y_pred = reg.predict(pca.transform(scaler.transform(df[columns].values)))
df['prediction'] = y_pred
df['error'] = df.prediction - df.weight
df['error_pct'] = df.error / df.weight
df['abs_error_pct'] = df.error_pct.abs()

model = {
    'mean': scaler.mean_,
    'std': scaler.scale_,
    'PCA_components': pca.components_,
    'reg_coef': reg.coef_,
    'reg_intercept': reg.intercept_,
    'body_parts': body_parts   
}



In [ ]:
pickle.dump(model, open('/root/data/alok/biomass_estimation/models/model_big_fish_4_eig.pkl', 'wb'))

In [ ]:
m = (df.gtsf_fish_identifier.str.contains('viking')) 
(df[m].prediction.mean() - df[m].weight.mean())/(df[m].weight.mean())

In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 10))

plt.scatter(df[~m]['weight'], df[~m]['prediction'], color='r')
plt.scatter(df[m]['weight'], df[m]['prediction'])
plt.xlabel('Ground Truth Weight')
plt.ylabel('Prediction')
plt.plot(range(10000), range(10000))


In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 10))
plt.scatter(df[mask]['weight'], df[mask]['prediction'])
plt.scatter(df[~mask]['weight'], df[~mask]['prediction'], color='r')
plt.xlabel('Ground Truth Weight')
plt.ylabel('Prediction')
plt.plot(range(10000), range(10000))


In [ ]:
session.query(GtsfDataCollection).filter(GtsfDataCollection.gtsf_fish_identifier == '190607010041_bolaks-mjanes').all()[0].ground_truth_metadata




In [ ]:
plt.figure(figsize=(15, 10))
plt.hist(df[(df.weight == 5571) & (df.length == 715)].prediction, bins=20)
plt.grid()
plt.axvline(x=5571, color='red', label='Ground Truth')
plt.xlabel('Predicted weight for fish ID = 190607010041_bolaks-mjanes')
plt.legend()
plt.show()

In [ ]:
df[(df.weight == 5571) & (df.length == 715)].prediction.mean()

In [ ]:
df[df.epoch == 1559904671235].prediction

In [ ]:
plt.hist(df.weight, bins=20)
plt.show()

In [ ]:
plt.plot(df.weight, df.length)
plt.show()

In [ ]:
m = (~mask) & (df.weight > 5000) & (df.prediction < 8000)
(df[m].prediction.mean() - df[m].weight.mean()) / (df[m].weight.mean())




In [ ]:
df[m].prediction.max()

In [ ]:
df[df.kfactor > 1.2].error.median()

In [ ]:
plt.grid()
plt.xlabel('K-factor')
plt.hist(df.kfactor, bins=20)

In [ ]:
N = 1000

# perform N-fold cross validation
abs_err_pcts = []
error_stds = []
for i in range(N):
    if i % 10 == 0:
        print(i)
    mask = generate_train_mask(df, train_frac=0.8)

    columns = ['{0}-{1}'.format(x, y) for x, y in list(combinations(list(range(8)), 2))]
    X_train = df.loc[mask, columns].values
    y_train = df.loc[mask, 'weight'].values
    X_test = df.loc[~mask, columns].values
    y_test = df.loc[~mask, 'weight'].values

    pca = PCA(n_components=min(X_train.shape[0], X_train.shape[1]))
    pca.fit(X_train)
    explained_variance_ratio = pca.explained_variance_ratio_.cumsum()
    idx = np.where(explained_variance_ratio > 0.99999)[0][0]

    pca = PCA(n_components=idx+1)
    pca.fit(X_train)
    X_train_modified = pca.transform(X_train)
    X_test_modified = pca.transform(X_test)
    reg = LinearRegression().fit(X_train_modified, y_train)


    y_pred = reg.predict(pca.transform(df[columns].values))
    df['prediction'] = y_pred
    df['error'] = df.prediction - df.weight
    df['error_pct'] = df.error / df.weight
    df['abs_error_pct'] = df.error_pct.abs()
    error_std = df.loc[~mask, 'error_pct'].std()
    error_stds.append(error_std)

    
    avg_biomass_err = df.loc[~mask, 'prediction'].mean() - df.loc[~mask, 'weight'].mean()
    abs_err_pct = abs(avg_biomass_err) / df.loc[~mask, 'weight'].mean()
    abs_err_pcts.append(abs_err_pct)
    
    



In [ ]:
data_sorted = sorted(list(abs_err_pcts))
p = 1.0 * np.arange(len(data_sorted)) / (len(data_sorted) - 1)
fig = plt.figure(figsize=(30, 7))
ax1 = fig.add_subplot(121)
ax1.plot(p, data_sorted)
ax1.set_xlabel('p')
ax1.set_ylabel('OOS error percentage')
plt.axvline(x=0.95, linestyle='--', color='red', label='p = 0.95')
plt.title('CDF of OOS errors (sample size = 250)')
plt.legend()
plt.grid()



<h1> Prod implementation test </h1>

In [ ]:
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, exc

def euclidean_distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5


def convert_to_world_point(x, y, d, parameters):
    """ from pixel coordinates to world coordinates """
    # get relevant parameters
    pixel_count_height = parameters["pixelCountHeight"]
    pixel_count_width = parameters["pixelCountWidth"]
    sensor_width = parameters["imageSensorWidth"]
    sensor_height = parameters["imageSensorHeight"]
    focal_length = parameters["focalLength"]

    image_center_x = pixel_count_height / 2.0
    image_center_y = pixel_count_width / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (sensor_height / pixel_count_height)
    sensor_z = px_z * (sensor_width / pixel_count_width)

    # now move to world coordinates
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return np.array([world_x, world_y, world_z])


def depth_from_disp(disp, parameters):
    """ calculate the depth of the point based on the disparity value """
    focal_length_pixel = parameters["focalLengthPixel"]

    baseline = parameters["baseline"]
    depth = focal_length_pixel * baseline / np.array(disp)
    return depth


def pixel2world(left_crop, right_crop, parameters):
    """2D pixel coordinates to 3D world coordinates"""

    # first create a dic with crop keypoints
    image_coordinates = {"leftCrop": {},
                         "rightCrop": {}}
    for keypoint in left_crop:
        name = keypoint["keypointType"]
        image_coordinates["leftCrop"][name] = [keypoint["xFrame"], keypoint["yFrame"]]
    for keypoint in right_crop:
        name = keypoint["keypointType"]
        image_coordinates["rightCrop"][name] = [keypoint["xFrame"], keypoint["yFrame"]]

    # then loop through the right crop keypoints and calculate the world coordinates
    world_coordinates = {}
    for keypoint in left_crop:
        name = keypoint["keypointType"]
        disparity = image_coordinates["leftCrop"][name][0] - image_coordinates["rightCrop"][name][0]
        depth = depth_from_disp(disparity, parameters)
        world_point = convert_to_world_point(image_coordinates["leftCrop"][name][1],
                                             image_coordinates["leftCrop"][name][0],
                                             depth,
                                             parameters)
        world_coordinates[name] = world_point
    return world_coordinates


def coord2biomass(world_keypoints, model):
    """from coordinates to biomass"""

    mean = model['mean']
    std= model['std']
    PCA_components = model['PCA_components']
    reg_coef = model['reg_coef']
    reg_intercept = model['reg_intercept']
    body_parts = model['body_parts']
    print(body_parts)

    # calculate pairwise distances for production coord
    # based on the exact ordering reflected in the body_parts
    # variable above

    pairwise_distances = []
    for i in range(len(body_parts)-1):
        for j in range(i+1, len(body_parts)):
            dist = euclidean_distance(world_keypoints[body_parts[i]], world_keypoints[body_parts[j]])
            pairwise_distances.append(dist)
            print(body_parts[i], body_parts[j], dist)
    print(pairwise_distances)
    
    interaction_values = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            dist1 = pairwise_distances[i]
            dist2 = pairwise_distances[j]
            interaction_values.append(dist1 * dist2)

    X = np.array(pairwise_distances + interaction_values)

    X_normalized = (X - model['mean']) / model['std']
    X_transformed = np.dot(X_normalized, model['PCA_components'].T)
    prediction = np.dot(X_transformed, reg_coef) + reg_intercept
    return prediction


In [ ]:
jsondata = json.load(open('sample_message.json'))
model = pickle.load(open('model.pkl', 'rb'))

In [ ]:
# annotation_id = jsondata["annotationId"]
parameters = jsondata["cameraParameters"]
right_crop = jsondata["rightCrop"]
left_crop = jsondata["leftCrop"]
site_id = jsondata["siteId"]
pen_id = jsondata["penId"]



# pixel coordinates to world coordinates
coordinates = pixel2world(left_crop, right_crop, parameters)
biomass = coord2biomass(coordinates, model)

In [ ]:
biomass

In [ ]:
plt.grid()
plt.hist(df_cache[(df_cache.kfactor > 0.3) & (df_cache.kfactor < 3)].kfactor, bins=30)

In [ ]:
euclidean_distance(coordinates['TAIL_NOTCH'], coordinates['UPPER_LIP'])

In [ ]:
df_cache[df_cache.kfactor > 3]